In [ ]:
!pip install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.6 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: flask
    Found existing installation: Flask 3.1.0
    Uninsta

In [ ]:
import open3d as o3d
import numpy as np
import json
import os
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt

In [ ]:
!unzip OneDrive_1_30-12-2024.zip

Archive:  OneDrive_1_30-12-2024.zip
 extracting: Crateva_seg5m.txt       
 extracting: Bauhinia_seg10.txt      
 extracting: Xanthostemon_seg2.txt   
 extracting: Delonix_seg6_nov.txt    
 extracting: Camphora_seg1.txt       


In [ ]:
import open3d as o3d
import numpy as np
import json
import os
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit

def load_txt_point_cloud(file_path):
    data = np.loadtxt(file_path, delimiter=' ')
    points = data[:, :3]  # x, y, z
    colors = data[:, 3:6] / 255.0  # r, g, b
    intensities = data[:, 6]  # i

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    return pcd

def compute_dbh(points, z_cut=1.3):
    min_z = np.min(points[:, 2])
    cut_points = points[np.abs(points[:, 2] - (min_z + z_cut)) < 0.05]

    def circle_model(xy, xc, yc, r):
        return (xy[:, 0] - xc)**2 + (xy[:, 1] - yc)**2 - r**2

    xy_points = cut_points[:, :2]
    initial_guess = [np.mean(xy_points[:, 0]), np.mean(xy_points[:, 1]), np.mean(np.linalg.norm(xy_points - np.mean(xy_points, axis=0), axis=1))]
    params, _ = curve_fit(circle_model, xy_points, np.zeros(len(xy_points)), p0=initial_guess)
    xc, yc, r = params

    return 2 * r  # Diameter

def process_point_cloud(file_path):
    pcd = load_txt_point_cloud(file_path)
    pcd = pcd.voxel_down_sample(voxel_size=0.05)
    bbox = pcd.get_axis_aligned_bounding_box()

    bbox_extent = bbox.get_extent()
    crown_spread = (bbox_extent[0] + bbox_extent[1]) / 2

    points = np.asarray(pcd.points)
    tree_height = np.max(points[:, 2]) - np.min(points[:, 2])

    dbh = compute_dbh(points)
    xy_points = points[:, :2]
    hull = ConvexHull(xy_points)

    hull_points = xy_points[hull.vertices]
    geojson = {
        "type": "Polygon",
        "coordinates": [hull_points.tolist()]
    }
    with open(file_path.replace(".txt", "_convex_hull.geojson"), "w") as f:
        json.dump(geojson, f)

    fig = plt.figure(figsize=(12, 5))

    ax1 = fig.add_subplot(131)
    ax1.scatter(points[:, 0], points[:, 1], c=points[:, 2], cmap='viridis', s=0.5)
    ax1.set_title('Top View')
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')

    ax3 = fig.add_subplot(132)
    ax3.scatter(points[:, 0], points[:, 2], c=points[:, 1], cmap='viridis', s=0.5)
    ax3.set_title('Side View')
    ax3.set_xlabel('X')
    ax3.set_ylabel('Z')

    ax4 = fig.add_subplot(133, projection='3d')
    ax4.scatter(points[:, 0], points[:, 1], points[:, 2], c=points[:, 2], cmap='viridis', s=0.5)
    ax4.set_title('3D View')
    ax4.set_xlabel('X')
    ax4.set_ylabel('Y')
    ax4.set_zlabel('Z')

    plt.tight_layout()
    plt.savefig(file_path.replace(".txt", "_views.png"))
    plt.close()

    print(f"Processing complete for {file_path}. Convex hull, metadata, and figures exported.")

    return {
        "file_name": os.path.basename(file_path),
        "crown_spread": crown_spread,
        "tree_height": tree_height,
        "dbh": dbh,
        "bounding_box": bbox_extent.tolist()
    }

def process_folder(folder_path):
    metadata_list = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_path, file_name)
            metadata = process_point_cloud(file_path)
            metadata_list.append(metadata)

    df = pd.DataFrame(metadata_list)
    df.to_csv(os.path.join(folder_path, "metadata_summary.csv"), index=False)
    print("Metadata summary exported as metadata_summary.csv")

folder_path = "/content/"
process_folder(folder_path)


Processing complete for /content/Xanthostemon_seg2.txt. Convex hull, metadata, and figures exported.
Processing complete for /content/Crateva_seg5m.txt. Convex hull, metadata, and figures exported.
Processing complete for /content/Camphora_seg1.txt. Convex hull, metadata, and figures exported.
Processing complete for /content/Delonix_seg6_nov.txt. Convex hull, metadata, and figures exported.
Processing complete for /content/Bauhinia_seg10.txt. Convex hull, metadata, and figures exported.
Metadata summary exported as metadata_summary.csv
